In [1]:
import xarray
from dask.distributed import Client, LocalCluster

cluster=  LocalCluster(n_workers=16, memory_limit='2GiB')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 80,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38821,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 80
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:34474,Total threads: 5
Dashboard: http://127.0.0.1:38462/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:33224,


In [2]:
import xarray as xr

ds = xarray.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/1959-2022-6h-512x256_equiangular_conservative.zarr',
    chunks=dict(lon=32,lat=32,time=4*365*5),
    storage_options=dict(token='anon'),
)
era5 = ds

/tmp/ipykernel_100932/2064813546.py:3: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 7300. This could degrade performance. Instead, consider rechunking after loading.
  ds = xarray.open_zarr(


In [5]:
era5

<xarray.Dataset> Size: 5TB
Dimensions:                                           (time: 92044,
                                                       longitude: 512,
                                                       latitude: 256, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 2kB ...
  * level                                             (level) int64 104B 50 ....
  * longitude                                         (longitude) float64 4kB ...
  * time                                              (time) datetime64[ns] 736kB ...
Data variables: (12/38)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 48GB dask.array<chunksize=(7300, 512, 256), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 48GB dask.array<chunksize=(7300, 512, 256), meta=np.ndarray>
    10m_wind_speed                                    (time, longitude, latitude) float32 48GB dask.array<chunksize=(7300, 512, 256), meta=np.ndarray>
    2m_temperature                                    (time, longitude, latitude) float32 48GB dask.array<chunksize=(7300, 512, 256), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (longitude, latitude) float32 524kB dask.array<chunksize=(512, 256), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (longitude, latitude) float32 524kB dask.array<chunksize=(512, 256), meta=np.ndarray>
    ...                                                ...
    type_of_high_vegetation                           (longitude, latitude) float32 524kB dask.array<chunksize=(512, 256), meta=np.ndarray>
    type_of_low_vegetation                            (longitude, latitude) float32 524kB dask.array<chunksize=(512, 256), meta=np.ndarray>
    u_component_of_wind                               (time, level, longitude, latitude) float32 627GB dask.array<chunksize=(7300, 13, 512, 256), meta=np.ndarray>
    v_component_of_wind                               (time, level, longitude, latitude) float32 627GB dask.array<chunksize=(7300, 13, 512, 256), meta=np.ndarray>
    vertical_velocity                                 (time, level, longitude, latitude) float32 627GB dask.array<chunksize=(7300, 13, 512, 256), meta=np.ndarray>
    wind_speed                                        (time, level, longitude, latitude) float32 627GB dask.array<chunksize=(7300, 13, 512, 256), meta=np.ndarray>

In [3]:
era5_features = era5[['u_component_of_wind','v_component_of_wind']].sel(level=850, drop=True)
era5_features['geopotential'] = era5['geopotential'].sel(level=500)
era5_features['total_precipitation_24hr'] = era5['total_precipitation_24hr']
era5_features = era5_features.transpose('time','latitude','longitude')
# era5_features['longitude'] = era5_features['longitude'].where(era5_features['longitude'] <180, era5_features['longitude']-360)
# era5_features = era5_features.sortby('longitude')

In [4]:
era5_features.nbytes/1e9

193.031001192

In [ ]:
era5_features.to_netcdf('/Data/gfi/users/rogui7909/era5_features.nc')

In [ ]:
era5_features.sel(latitude=slice(0,90))

In [ ]:
era5_features.isel(latitude=range(128))

In [6]:
era5_features = xr.concat([era5_features.sel(longitude=slice(-180,0)), era5_features.sel(longitude=slice(0,180))], dim='longitude')

KeyError: 180

In [ ]:
era5_features.isel(time=0).sel(latitude=slice(0,90)).u_component_of_wind.plot()

In [4]:
era5_rain = era5['total_precipitation_24hr']

In [ ]:
era5_features_daily = era5_features.resample(time='D').mean(keep_attrs=True)

In [ ]:
era5_features_daily.isel(time=0).u_component_of_wind.plot()

2025-01-10 16:22:57,160 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/felles_gfi/felles_gfi_users/rogui7909/miniforge3/envs/robin/lib/python3.12/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/felles_gfi/felles_gfi_users/rogui7909/miniforge3/envs/robin/lib/python3.12/site-packages/distributed/worker.py", line 1269, in heartbeat
    response = await retry_operation(
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/felles_gfi/felles_gfi_users/rogui7909/miniforge3/envs/robin/lib/python3.12/site-packages/distributed/utils_comm.py", line 441, in retry_operation
    return await retry(
           ^^^^^^^^^^